In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "Kline.csv"), delim=';');
size(d) # Should be 10x5

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map(x -> log(x), d[:population]);

New col contact_high, set binary values 1/0 if high/low contact

In [3]:
d[:contact_high] = map(x -> ifelse(x=="high", 1, 0), d[:contact]);

This is supposed to be a "bad" model since we take non-centered data for the
predictor log_pop

In [4]:
@model m10_10stan(total_tools, log_pop, contact_high) = begin
    α ~ Normal(0, 100)
    βp ~ Normal(0, 1)
    βc ~ Normal(0, 1)
    βpc ~ Normal(0, 1)

    for i ∈ 1:length(total_tools)
        λ = exp(α + βp*log_pop[i] + βc*contact_high[i] +
            βpc*contact_high[i]*log_pop[i])
        total_tools[i] ~ Poisson(λ)
    end
end;

posterior = sample(m10_10stan(d[:total_tools], d[:log_pop],
    d[:contact_high]), Turing.NUTS(2000, 1000, 0.95));
# Fix the inclusion of adaptation samples
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/c

Rethinking result

In [5]:
m_10_10t_result = "
     mean   sd  5.5% 94.5% n_eff Rhat
 a    0.94 0.37  0.36  1.53  3379    1
 bp   0.26 0.04  0.21  0.32  3347    1
 bc  -0.08 0.84 -1.41  1.23  2950    1
 bpc  0.04 0.09 -0.10  0.19  2907    1
";

Describe the draws

In [6]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                  SD                        Naive SE               MCSE         ESS    
       α   1.009786829  0.3394524954761143908932297 0.01073443043132524254978932 0.0419021607   65.627396
  lf_num   0.000000000  0.0000000000000000000000000 0.00000000000000000000000000 0.0000000000         NaN
      βc  -0.332783856  0.7178033694645147289037368 0.02269893559651224579365447 0.1644604967   19.049685
     βpc   0.068297990  0.0791787637764899826509080 0.00250385235850143555644265 0.0180946915   19.147632
 elapsed   0.022063971  0.0125735563901941266307105 0.00039761076481578254719282 0.0005000883  632.153985
 epsilon   0.004748676  0.0000000000000000052067745 0.00000000000000000016465267 0.0000000000 1000.000000
      βp   0.257285258  0.0328112637678237076266363 0.00103758326414881080559938 0.0039366333   69.469792
      lp -45.682883568  1.3701808426640988

End of m_10_10t.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*